In [1]:
# set up logging
import logging
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s -    %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)

In [2]:
# make deterministic
from mingpt.utils import set_seed
set_seed(42)

In [3]:
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F

In [4]:
import math
from torch.utils.data import Dataset


class CharDataset(Dataset):

    def __init__(self, data, block_size):
        chars = sorted(list(set(data)))
        data_size, vocab_size = len(data), len(chars)
        print(f"data has {data_size:d} characters, {vocab_size:d} unique.")

        self.stoi = { ch: i for i, ch in enumerate(chars) }
        self.itos = { i: ch for i, ch in enumerate(chars) }
        self.block_size = block_size
        self.vocab_size = vocab_size
        self.data = data
    
    def __len__(self):
        return len(self.data) - self.block_size
    
    def __getitem__(self, idx):
        # grab a chunk of (block_size + 1) characters from the data
        chunk = self.data[idx:idx+self.block_size+1]
        # encode every character to an integer
        dix = [self.stoi[s] for s in chunk]
        """
        arrange data and targets so that the first i elements of x
        will be asked to predict the i-th element of y. Notice that
        the eventual language model will actually make block_size
        individual predictions at the same time based on this data,
        so we are being clever and amortizing the cost of the forward
        pass of the network. So for example if block_size is 4, then
        we could e.g. sample a chunk of text "hello", the integers in
        x will correspond to "hell" and in y will be "ello". This will
        then actually "multitask" 4 separate examples at the same time
        in the language model:
        - given just "h", please predict "e" as next
        - given "he" please predict "l" next
        - given "hel" predict "l" next
        - given "hell" predict "o" next
        
        In addition, because the DataLoader will create batches of examples,
        every forward/backward pass during traning will simultaneously train
        a LOT of predictions, amortizing a lot of computation. In particular,
        for a batched input of integers X (B, T) where B is batch size and
        T is block_size and Y (B, T), the network will during training be
        simultaneously training to make B*T predictions, all at once! Of course,
        at test time we can paralellize across batch B, but unlike during training
        we cannot parallelize across the time dimension T - we have to run
        a forward pass of the network to recover the next single character of the 
        sequence along each batch dimension, and repeatedly always feed in a next
        character to get the next one.
        
        So yes there is a big asymmetry between train/test time of autoregressive
        models. During training we can go B*T at a time with every forward pass,
        but during test time we can only go B at a time, T times, with T forward 
        passes.
        """
        x = torch.tensor(dix[:-1], dtype=torch.long)
        y = torch.tensor(dix[1:], dtype=torch.long)
        return x, y

In [5]:
block_size = 128  # spacial extent of the model for its context

In [6]:
text = open("input.txt", "r").read()  # don't worry we won't run out of file handles
train_dataset = CharDataset(text, block_size)  # one line of poem is roughly 50 characters

data has 1115394 characters, 65 unique.


In [7]:
from mingpt.model import GPT, GPTConfig


mconf = GPTConfig(
    train_dataset.vocab_size,
    train_dataset.block_size,
    n_layer=4,
    n_head=8,
    n_embd=512,
)
model = GPT(mconf)

02/05/2022 08:48:32 - INFO - mingpt.model -    number of parameters: 12742656


In [8]:
from mingpt.trainer import Trainer, TrainerConfig

# initialize a trainer instance and kick off training
tconf = TrainerConfig(
    max_epochs=2,
    batch_size=512,
    learning_rate=6e-4,
    lr_decay=True,
    warmup_tokens=512*20,
    final_tokens=2*len(train_dataset)*block_size,
    num_workers=4,
)
trainer = Trainer(model, train_dataset, None, tconf)
trainer.train()

epoch 1 iter 2178: train loss 0.67980, lr 3.000169e-04: 100%|██████████| 2179/2179 [12:02<00:00,  3.02it/s]
epoch 2 iter 2178: train loss 0.38349, lr 6.000000e-05: 100%|██████████| 2179/2179 [12:03<00:00,  3.01it/s]


In [9]:
# alright, let's sample some character-level Shaespeare
from mingpt.utils import sample

context = "O God, O God!"
x = torch.tensor([train_dataset.stoi[s] for s in context], dtype=torch.long)[None, ...].to(trainer.device)
y = sample(model, x, 2000, temperature=1.0, sample=True, top_k=10)[0]
completion = ''.join([train_dataset.itos[int(i)] for i in y])
print(completion)

O God, O God! that e'er this thine!

QUEEN MARGARET:
I call thee not, king; let's fight with grief and be
That thou dost contrary any more.

CLARENCE:
O, no, he loves me, and he holds me dear:
Go you to him from me.

First Murderer:
Back, and what thy head is great.
Strive County Paris! why do you wring your hands?

VALERIA:
Not out of doors!

VOLUMNIA:
Should we be silent and not speak, and speak apace.

CORIOLANUS:
I dare be sworn
To give me the present army. Answer
me like some words in Padua short to the garden,
That makes his opening warm scars to see him:
He was done so by, and so fair a show!

KING RICHARD III:
He am glad on here of this: God defend me  he rest.

QUEEN ELIZABETH:
How fares the prince, as he is just.

KING RICHARD III:
So, to the dire and heights of all at
With mine eyes to save her brother's life.
Master, the Volsces have struck deadly blown,
The grass whereon the beggar's traitor
Which in their indifferent ruin.

CORIOLANUS:
To moment.

AUFIDIUS:
I understand t